# Реализация CatBoostCV

In [13]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import warnings

from catboost import CatBoostRegressor
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from utils.data_manager import DataManager
from utils.model_manager import ModelManager
from utils.syth_generator_gaussian import CombinedSyntheticGenerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# --- Глобально включаем вывод Pandas для всех трансформеров ---
# (Можно применять и к отдельным трансформерам/пайплайнам .set_output(transform="pandas"))
set_config(transform_output = "pandas")

In [15]:
dm = DataManager()
mm = ModelManager()

# Отключаем автологгирование, чтобы использовать ручное
mlflow.sklearn.autolog(disable=True)
warnings.filterwarnings("ignore", module="mlflow")  # Игнорируем предупреждения MLflow


In [16]:
RANDOM_STATE = 42
N_FOLDS = 5  # Например, 5 или 10

## 1. Загрузка данных

In [17]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')
train_data.shape

(1460, 81)

## 2. Предобработка данных 

In [18]:
# Определение колонок для удаления
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)

In [19]:
# Разделение на X / y
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)
X_test = test_data.copy()
print(X_test.shape)

(1460, 80) (1460,)
(1459, 80)


In [20]:
X.drop(columns=bad_columns, inplace=True)
X_test.drop(columns=bad_columns, inplace=True)

In [21]:
def make_feature_eng_great_again(train_X_in, test_X_in):
    """Хелпер, который создает фичи, логарифмирует и выравнивает колонки."""
    # Работаем с копиями, чтобы не изменять оригинальные X, X_test вне функции
    train_X = train_X_in.copy()
    test_X = test_X_in.copy()

    # Словарь качественных признаков
    quality_dict = {'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}

    def create_features(df):
        # Interactions (с проверкой на наличие колонок)
        if 'Neighborhood' in df.columns and 'MSZoning' in df.columns:
            df['Neighborhood_Zoning'] = df['Neighborhood'].astype(str) + '_' + df['MSZoning'].astype(str)
            # df.drop(columns=['Neighborhood', 'MSZoning'], inplace=True)  # mean CV RMSLE улучшился на 0.008, oof rmse улучшился на 0.0011; KAGGLE LB УПАЛ!!!
        if 'SaleType' in df.columns and 'SaleCondition' in df.columns:
            df['SaleType_Condition'] = df['SaleType'].astype(str) + '_' + df['SaleCondition'].astype(str)
            # df.drop(columns=['SaleType', 'SaleCondition'], inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!

        # Quality Score
        df['TotalQualScore'] = 0
        quality_cols = ['ExterQual', 'KitchenQual', 'BsmtQual', 'HeatingQC', 'GarageQual', 'FireplaceQu']
        for col in quality_cols:
            if col in df.columns:
                 df['TotalQualScore'] += df[col].map(quality_dict).fillna(0)
        # df.drop(columns=quality_cols, inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!

        # Porch/Deck Area and Flags
        df['PorchDeckArea'] = 0
        porch_cols = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
        for col in porch_cols:
             if col in df.columns:
                df['PorchDeckArea'] += df[col].fillna(0)
        # df.drop(columns=porch_cols, inplace=True)  # Mean CV RMSE улучшился на 0.0008, oof rmse улучшился на 0.0006; KAGGLE LB УПАЛ!!!

        if 'Fireplaces' in df.columns:
            df['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)
            # df.drop(columns=['Fireplaces'], inplace=True)  # !!! mean cv rmse улучшился на 0.0023, oof rmse улучшился на 0.0031. fold 3 проблемный очень хорошо улучшился без этой фичи; KAGGLE LB УПАЛ!!!
        if 'GarageType' in df.columns:
            df['HasGarage'] = (~df['GarageType'].isna()).astype(int)
            # df.drop(columns=['GarageType'], inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!
        if 'Fence' in df.columns:
            df['HasFence'] = (~df['Fence'].isna()).astype(int)
            # df.drop(columns=['Fence'], inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!
        df['HasPorchDeck'] = (df['PorchDeckArea'] > 0).astype(int)
        # df.drop(columns=['PorchDeckArea'], inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!

        return df

    def log_features(df, cols_to_log_list):  # Принимает СПИСОК колонок
        print(f"Applying log1p to: {cols_to_log_list}")
        for col_name in cols_to_log_list:
            if col_name in df.columns:
                # Добавим проверку на отрицательные значения перед логарифмированием
                if (df[col_name] < 0).any():
                     print(f"Warning: Column {col_name} contains negative values. Skipping log1p.")
                else:
                    df[col_name] = np.log1p(df[col_name])
            else:
                print(f"Warning: Column {col_name} not found in DF during log transform.")
        return df

    # 1. Создаем фичи
    train_X = create_features(train_X)
    test_X = create_features(test_X)
    print("Features created.")

    # 2. Определяем колонки для логарифмирования (ТОЛЬКО по трейну)
    numeric_cols = train_X.select_dtypes(include=np.number).columns
    skew_values = train_X[numeric_cols].skew()
    # Используем .index.tolist() чтобы получить список имен
    cols_to_log_list = skew_values[skew_values > 1].index.tolist()
    print(f"Columns identified for logging: {cols_to_log_list}")

    # # 3. Логарифмируем (используя ОДИН и тот же список)
    # train_X = log_features(train_X, cols_to_log_list)
    # test_X = log_features(test_X, cols_to_log_list)
    # print("Log transform applied.")

    # 4. Согласуем и сортируем колонки ПОСЛЕ всех манипуляций
    final_feature_cols = sorted(train_X.columns.tolist()) # Сортируем для стабильности
    train_X = train_X[final_feature_cols]
    test_X = test_X.reindex(columns=final_feature_cols, fill_value=0)
    print("Columns aligned and sorted.")

    return train_X, test_X

In [22]:
# Вызываем функцию с правильными данными (X, X_test)
X, X_test = make_feature_eng_great_again(X, X_test)

print("\nProcessing complete. Final shapes:")
print(f"X_processed: {X.shape}")
print(f"X_test_processed: {X_test.shape}")
print("\nExample processed X:")
print(X.head())

Features created.
Columns identified for logging: ['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PorchDeckArea', 'HasFence']
Columns aligned and sorted.

Processing complete. Final shapes:
X_processed: (1460, 81)
X_test_processed: (1459, 81)

Example processed X:
   1stFlrSF  2ndFlrSF  3SsnPorch Alley  BedroomAbvGr BldgType BsmtCond  \
0       856       854          0   NaN             3     1Fam       TA   
1      1262         0          0   NaN             3     1Fam       TA   
2       920       866          0   NaN             3     1Fam       TA   
3       961       756          0   NaN             3     1Fam       Gd   
4      1145      1053          0   NaN             4     1Fam       TA   

  BsmtExposure  BsmtFinSF1  BsmtFinSF2  ... ScreenPorch Street  TotRmsAbvGrd  \


In [23]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

## 3. Обучаем модель с CV и корректируя данные

In [49]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    # ('scaler', StandardScaler())  # разницы не дает 
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# --- Объединяем препроцессоры ---
preprocessor = ColumnTransformer(
    transformers=[
        # Применяем к исходным числовым колонкам
        ('num', numeric_transformer, numeric_columns),
        # Применяем к исходным категориальным колонкам
        ('cat', categorical_transformer, non_numeric_columns)
    ],
    remainder='drop',   # 'passthrough' сохранит полиномиальные и другие колонки, которые не были ни числовыми, ни категориальными ИЗНАЧАЛЬНО
    verbose_feature_names_out=False  # Чтобы имена колонок не менялись на 'num__colname' и т.д.
)

X = preprocessor.fit_transform(X)
X_test = preprocessor.transform(X_test)

In [42]:
def print_cv_summary(cv_data):
    print("CV Results DataFrame Head:")
    print(cv_data.head()) # Печатаем начало таблицы для проверки имен колонок
    print("\nAvailable columns:", cv_data.columns.tolist()) # Печатаем все имена колонок
    print("\n")

    # --- ИСПРАВЛЕНИЕ ---
    # Используем имена колонок для RMSE, т.к. loss_function='RMSE'
    metric_mean_col = 'test-RMSE-mean'
    metric_std_col = 'test-RMSE-std'

    # Проверка на случай, если колонки все же не создались (маловероятно, но надежнее)
    if metric_mean_col not in cv_data.columns:
        print(f"ОШИБКА: Колонка '{metric_mean_col}' не найдена в результатах CV!")
        return # Выходим из функции, если нет нужной колонки

    best_value = cv_data[metric_mean_col].min()
    # Используем idxmin() для pandas Series - надежнее, чем argmin()
    best_iter = cv_data[metric_mean_col].idxmin()

    # Получаем стандартное отклонение, проверяя наличие колонки
    std_value = 0.0 # Значение по умолчанию, если колонка std отсутствует
    if metric_std_col in cv_data.columns:
         # Доступ по индексу best_iter, который вернул idxmin()
        std_value = cv_data[metric_std_col][best_iter]
    else:
        print(f"Предупреждение: Колонка '{metric_std_col}' не найдена. Std будет 0.")


    print('Best validation RMSE score : {:.4f} ± {:.4f} on step {}'.format(
        best_value,
        std_value, # Используем полученное или дефолтное значение std
        best_iter)
    )

In [48]:
from catboost import  cv, Pool

# --- Логарифмирование целевой переменной ---
y_log = np.log1p(y)  # Т.к. нет RMSLE потери у loss_function catboost.cv

train_pool = Pool(data=X, 
                  label=y_log,
                  cat_features=non_numeric_columns.values, 
                  has_header=True
                  )

# parameters for training inside cv:
params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
}

cv_data = cv(
    params=params,
    pool=train_pool,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    plot=False,
    stratified=False,
    verbose=False
)

print_cv_summary(cv_data)

Training on fold [0/5]

bestTest = 0.1883383887
bestIteration = 994

Training on fold [1/5]

bestTest = 0.191445652
bestIteration = 996

Training on fold [2/5]

bestTest = 0.1831752595
bestIteration = 994

Training on fold [3/5]

bestTest = 0.1853328536
bestIteration = 998

Training on fold [4/5]

bestTest = 0.1615958298
bestIteration = 997

CV Results DataFrame Head:
   iterations  test-RMSE-mean  test-RMSE-std  train-RMSE-mean  train-RMSE-std
0           0       11.449492       0.016265        11.447650        0.008519
1           1       10.897370       0.024148        10.896375        0.017496
2           2       10.375870       0.032902        10.373011        0.019943
3           3        9.874257       0.032008         9.870556        0.023149
4           4        9.396932       0.030249         9.396275        0.023939

Available columns: ['iterations', 'test-RMSE-mean', 'test-RMSE-std', 'train-RMSE-mean', 'train-RMSE-std']


Best validation RMSE score : 0.1820 ± 0.0118 on step

In [51]:
# 1. Получаем оптимальное количество итераций из результатов CV
metric_mean_col = 'test-RMSE-mean'
if metric_mean_col in cv_data.columns:
    best_iter_idx = cv_data[metric_mean_col].idxmin()
    optimal_iterations = best_iter_idx + 1
    print(f"\nОптимальное количество итераций по результатам CV: {optimal_iterations}")
else:
    print(f"Не удалось найти колонку {metric_mean_col} в cv_data. Используем дефолтное количество итераций.")
    optimal_iterations = params.get('iterations', 1000) # Берем из исходных params

# 2. Определяем параметры для финальной модели
final_params = params.copy()
final_params['iterations'] = optimal_iterations
final_params['random_seed'] = RANDOM_STATE
# Убери custom_loss если он был в params
if 'custom_loss' in final_params: del final_params['custom_loss']

print("\nПараметры для финальной модели:")
print(final_params)

# 3. Инициализируем и обучаем финальную модель
final_model = CatBoostRegressor(**final_params)

print("\nОбучение финальной модели на всем train_pool...")
# Обучаем на том же train_pool (который содержит X и y_log)
final_model.fit(train_pool, verbose=100)
print("Обучение завершено.")

# 4. Делаем предсказания на тестовых данных (X_test)
#    Убедись, что X_test прошел ТОЧНО ТАКУЮ ЖЕ обработку NaN в категориальных колонках, как X
print("\nСоздание предсказаний для X_test...")
predictions_log = final_model.predict(X_test) # Предсказания будут в лог. масштабе
print("Предсказания (в лог. масштабе) созданы.")

# 5. Пост-обработка предсказаний: ОБРАТНОЕ ПРЕОБРАЗОВАНИЕ (ОБЯЗАТЕЛЬНО!)
print("Применение обратного преобразования np.expm1() к предсказаниям...")
predictions = np.expm1(predictions_log) # <--- Вот этот шаг теперь обязателен!
print("Обратное преобразование завершено.")

# Проверка на отрицательные значения (цены не могут быть отрицательными)
predictions[predictions < 0] = 0 # Заменяем возможные отриц. значения на 0

# 6. Создание файла для сабмишна
print("\nФормирование файла для сабмишна...")
# Убедись, что test_data - это исходный файл test.csv, загруженный в начале
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': predictions
})

# Сохраняем файл
submission_file = 'final_submission_Clean_catboost_cv_log_v1.csv'
submission.to_csv(submission_file, index=False)
print(f"Файл для сабмишна сохранен как: {submission_file}")
print(submission.head())




Оптимальное количество итераций по результатам CV: 997

Параметры для финальной модели:
{'iterations': 997, 'learning_rate': 0.05, 'depth': 6, 'loss_function': 'RMSE', 'random_seed': 42}

Обучение финальной модели на всем train_pool...
0:	learn: 0.3855520	total: 17.7ms	remaining: 17.6s
100:	learn: 0.1126572	total: 956ms	remaining: 8.48s
200:	learn: 0.0950676	total: 1.83s	remaining: 7.24s
300:	learn: 0.0862630	total: 2.87s	remaining: 6.65s
400:	learn: 0.0794245	total: 3.78s	remaining: 5.62s
500:	learn: 0.0718999	total: 4.65s	remaining: 4.61s
600:	learn: 0.0665271	total: 5.41s	remaining: 3.56s
700:	learn: 0.0611366	total: 6.28s	remaining: 2.65s
800:	learn: 0.0562032	total: 7.08s	remaining: 1.73s
900:	learn: 0.0520739	total: 8.03s	remaining: 855ms
996:	learn: 0.0488397	total: 8.79s	remaining: 0us
Обучение завершено.

Создание предсказаний для X_test...
Предсказания (в лог. масштабе) созданы.
Применение обратного преобразования np.expm1() к предсказаниям...
Обратное преобразование заверше

# ИТОГО
Никакой магии, дефолтный cv катбуста дал такие же показатели.